In [12]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [34]:
df = pd.read_csv("/home/dom/Documents/MPhys/TheGrandTour/wine_data.txt", sep="\t");
data = np.array(df);
data = np.delete(data, 0, 0)
data = data.astype(float)
data = np.swapaxes(data,0,1)

classification = data[13]
data = np.delete(data, 13, axis=0)
# Normalizes the data        
for i in range(0, np.shape(data)[0]):
    data[i,:] = (data[i,:] / np.ndarray.max(data[i,:])) * 2 - 1


In [16]:
stepSize = 0.01
nSteps = 10000

def getAlpha(d):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.
    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d
    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane
    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane
    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    Alpha = getAlpha(d)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData, tBeta


tData, tBeta = GrandTour(data, nSteps)


In [35]:
targetData = np.zeros((len(tData[0][0]), 3))
for counter, i in enumerate(classification):
    targetData[counter][int(i-1)] = 1
targetData

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [42]:
x = torch.from_numpy(data)
y = torch.from_numpy(targetData)
print(x.size()[0])
print(y.size()[1])
x.transpose_(0, 1)


13
3


In [54]:
print(x.size())
x.transpose_(0, 1)
x.size()

torch.Size([177, 13])

In [52]:
print(x[0])
print(x[1])
print(x.size())
print(y.size())

tensor([0.7802, 0.7748, 0.9380, 0.7856, 0.9150, 0.9407, 0.8962, 1.0000, 0.8692,
        0.9016, 0.9042, 0.8543, 0.9892, 0.9393, 0.8382, 0.9285, 0.8651, 0.9137,
        0.8395, 0.8962, 0.7438, 0.8490, 0.7330, 0.8206, 0.7599, 0.8058, 0.7937,
        0.8705, 0.8908, 0.8517, 0.8314, 0.8449, 0.8557, 0.8220, 0.8179, 0.7910,
        0.7599, 0.7626, 0.9177, 0.8287, 0.8085, 0.8719, 0.7856, 0.7599, 0.9164,
        0.9393, 0.8746, 0.9016, 0.8800, 0.7599, 0.8651, 0.8638, 0.8570, 0.8530,
        0.8287, 0.9177, 0.7923, 0.8503, 0.6682, 0.6628, 0.7047, 0.8436, 0.6682,
        0.6413, 0.6682, 0.7680, 0.6682, 0.7991, 0.6467, 0.6575, 0.8692, 0.8193,
        0.7519, 0.6129, 0.5725, 0.7572, 0.5968, 0.6628, 0.7127, 0.6183, 0.7154,
        0.6291, 0.7599, 0.5968, 0.7087, 0.6399, 0.5711, 0.5698, 0.6291, 0.6291,
        0.6183, 0.7114, 0.6575, 0.5671, 0.6817, 0.5927, 0.6575, 0.6682, 0.6575,
        0.6291, 0.6993, 0.6642, 0.5941, 0.6871, 0.6750, 0.6521, 0.7154, 0.6480,
        0.5657, 0.5455, 0.6885, 0.5860, 

In [74]:
class Neural_Network(nn.Module):
    
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = x.size()[1]
        self.outputSize = y.size()[1]
        self.hiddenSize = 20
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
        
    def forward(self, X):
        #print(X.size())
        #print(self.W1.size())
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self, X):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(X))
        print ("Output: \n" + str(self.forward(X)))

In [77]:
NN = Neural_Network()
torch.set_default_tensor_type('torch.FloatTensor')


In [83]:
NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(x)) ** 2).detach().item()))  # mean sum squared loss
    NN.train(x, y)
NN.saveWeights(NN)
NN.predict()

#0 Loss: 0.36123135685920715
#1 Loss: 0.34010592103004456
#2 Loss: 0.3992437422275543
#3 Loss: 0.39924368262290955
#4 Loss: 0.39924365282058716
#5 Loss: 0.3992435932159424
#6 Loss: 0.39924356341362
#7 Loss: 0.3992435038089752
#8 Loss: 0.39924344420433044
#9 Loss: 0.39924338459968567
#10 Loss: 0.3992433547973633
#11 Loss: 0.3992433249950409
#12 Loss: 0.39924323558807373
#13 Loss: 0.39924317598342896
#14 Loss: 0.3992430865764618
#15 Loss: 0.399243026971817
#16 Loss: 0.39924296736717224
#17 Loss: 0.39924290776252747
#18 Loss: 0.3992428183555603
#19 Loss: 0.3992427587509155
#20 Loss: 0.39924269914627075
#21 Loss: 0.3992426097393036
#22 Loss: 0.3992425799369812
#23 Loss: 0.39924249053001404
#24 Loss: 0.3992424011230469
#25 Loss: 0.3992423415184021
#26 Loss: 0.39924222230911255
#27 Loss: 0.3992421329021454
#28 Loss: 0.3992420434951782
#29 Loss: 0.39924192428588867
#30 Loss: 0.3992418348789215
#31 Loss: 0.39924174547195435
#32 Loss: 0.3992415964603424
#33 Loss: 0.39924144744873047
#34 Loss: 0

#513 Loss: 0.19962994754314423
#514 Loss: 0.19962991774082184
#515 Loss: 0.19962990283966064
#516 Loss: 0.19962990283966064
#517 Loss: 0.19962988793849945
#518 Loss: 0.19962987303733826
#519 Loss: 0.19962987303733826
#520 Loss: 0.19962985813617706
#521 Loss: 0.19962985813617706
#522 Loss: 0.19962984323501587
#523 Loss: 0.19962984323501587
#524 Loss: 0.19962984323501587
#525 Loss: 0.19962981343269348
#526 Loss: 0.1996297985315323
#527 Loss: 0.1996297985315323
#528 Loss: 0.1996297836303711
#529 Loss: 0.1996297687292099
#530 Loss: 0.1996297538280487
#531 Loss: 0.1996297538280487
#532 Loss: 0.1996297538280487
#533 Loss: 0.1996297389268875
#534 Loss: 0.19962972402572632
#535 Loss: 0.19962972402572632
#536 Loss: 0.19962972402572632
#537 Loss: 0.19962970912456512
#538 Loss: 0.19962969422340393
#539 Loss: 0.19962969422340393
#540 Loss: 0.19962967932224274
#541 Loss: 0.19962966442108154
#542 Loss: 0.19962964951992035
#543 Loss: 0.19962961971759796
#544 Loss: 0.19962961971759796
#545 Loss: 0.199

#996 Loss: 0.19962716102600098
#997 Loss: 0.19962716102600098
#998 Loss: 0.19962714612483978
#999 Loss: 0.19962714612483978
Predicted data based on trained weights: 


NameError: name 'xPredicted' is not defined

In [69]:
x = x.float()
y = y.float()
x.type()


'torch.FloatTensor'